In [ ]:
import numpy as np 
import pandas as pd 
import nltk
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import AutoTokenizer
import torch

In [ ]:
training_df = pd.read_csv("C:/PROJECTS/Abstractive Text Summarization/dataset/cnn_dailymail/train.csv") 


In [ ]:
MAX_LEN = 512
SUMMARY_LEN = 150
TRAINNING_SIZE = 5000

training_df = training_df.iloc[0:TRAINNING_SIZE,:].copy()

training_article_ls = list(training_df['article'])
training_highlight_ls = list(training_df['highlights'])

del training_df

In [ ]:
df = pd.DataFrame(columns=['target_text','source_text'])
df['target_text'] = training_highlight_ls
df['source_text'] = ['summarize: '+item for item in training_article_ls]

Run Pre-trained PEGASUS on CNN/DailyMail dataset.

In [ ]:
model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
#tokenizer = PegasusTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(df['source_text'], truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
assert (tgt_text == df['target_text'])